# [Day 23 - Crab Cups](https://adventofcode.com/2020/day/23)
## Part 1

In [1]:
# Perform one move, returning new start, and reordered cups
def cups_move(start:int, cups):
    # First cup position
    i = cups.index(start)

    # Pickup next 3 cups
    cups.rotate(-(i+1))
    pickup = [cups.popleft() for i in range(3)]
    
    # Get destination 
    #sorted_cups = sorted(cups)
    #dest = sorted_cups[sorted_cups.index(start)-1]
    try:
        dest = max([c for c in cups if c < start])
    except ValueError:
        dest = max(cups)

    # Replace pickup cups
    dest_i = cups.index(dest)
    cups.rotate(-(dest_i+1))
    cups.extend(pickup)

    # Next current cup
    i = cups.index(start)
    cups.rotate(-(i+1))
    start = cups[0]

    return start,cups

In [2]:
from collections import deque
from copy import deepcopy

input = list(map(int,open("inputs/23-input.txt").read().splitlines()[0]))

cups = deque(deepcopy(input))
#cups = deque([3,8,9,1,2,5,4,6,7])

# Start with 1st cup in list
start = cups[0]
# Execute 100 moves
for i in range(100):
    #print(i+1, start, cups)
    start, cups = cups_move(start, cups)

# Get formatted answer (digits in cups, clockwise from 1)
cups.rotate(-cups.index(1))
cups.popleft()
"".join(str(i) for i in cups)

'59374826'

This won't work for Part 2 because it's slow as balls!

`deque` is not ideal as there's no way around having to sort it or search and index it, which doesn't scale well for 1,000,000 values.

Let's try using a dictionary to create a linked list...


In [3]:
# Print cups as string
def print_cups(d, start):
    next_cup = d[start]
    l = str(start)
    while next_cup != start:
        l += str(next_cup)
        next_cup = d[next_cup]
    #print(l)
    return l

# Perform one move, returning new start, and reordered cups
def cups_move_fast(start:int, d:dict):
    # First cup
    first = d[start]

    # Pickup next 3 cups
    a = d[first]
    b = d[a]
    c = d[b]

    # Link first to next cup after pickup
    d[first] = d[c]

    # Get destination
    dest = first - 1
    while True:
        # Loop back round when dest hits zero
        if dest == 0:
            dest = max_cup
        # Destination not in remaining cups
        if dest in [a, b, c, first]:
            dest -= 1
        else:
            break

    # Move pickup cards to destination
    dest_next = d[dest]
    d[dest] = a
    d[c] = dest_next

    # New start
    start = first

    return start, d 

In [4]:
cups = deepcopy(input)
#cups = [3,8,9,1,2,5,4,6,7]

# Circular linked list dictionary
d = {cups[i]:cups[i+1] for i in range(-1,len(cups)-1)}

# Start with 1st cup in list
start = cups[-1]
max_cup = max(cups)
# Execute 100 moves
for i in range(100):
    #print(i+1, start, print_cups(d,d[start]))
    start, d = cups_move_fast(start, d)

# Get formatted answer (digits in cups, clockwise from 1)
print_cups(d,1)[1:]

'59374826'

## Part 2
Now should work with same method

In [5]:
cups = deepcopy(input)
#cups = [3,8,9,1,2,5,4,6,7]

# Fill in cups to length 1,000,000
long_cups = cups + [i+1 for i in range(len(cups),1000000)]

# Circular linked list dictionary
d = {long_cups[i]:long_cups[i+1] for i in range(-1,len(long_cups)-1)}

# Start with 1st cup in list
start = long_cups[-1]
max_cup = max(long_cups)
# Execute 1,000,000 moves
for i in range(10000000):
    #print(i+1, start, print_cups(d,d[start]))
    start, d = cups_move_fast(start, d)

# Multiply 2 cups after cup 1
a = d[1]
b = d[a]
a*b

66878091588